In [1]:
%load_ext autoreload
%autoreload 2

In [130]:
from mvf_bto.data_loading import load_data
from mvf_bto.constants import * 
from mvf_bto.models.baseline_lstm import BaselineLSTM
from mvf_bto.preprocessing import create_discharge_inputs
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.metrics import MeanSquaredError
from tensorflow.keras.models import load_model
from scipy.interpolate import interp1d

import numpy as np
import pandas as pd
import plotly
import plotly.graph_objects as go
import plotly.express as px

## Loading Data

In [3]:
# data_path = "/Users/anoushkabhutani/PycharmProjects/10701-mvf-bto/data/2017-05-12_batchdata_updated_struct_errorcorrect.mat"
data_path = "/Users/mac/Desktop/CMU/10701MachineLearning/project/10701-mvf-bto-backup/data/2017-05-12_batchdata_updated_struct_errorcorrect.mat"

In [4]:
data = load_data(file_path=data_path, num_cells=11)

100%|██████████| 11/11 [01:30<00:00,  8.21s/it]


In [5]:
import os
for filename in os.listdir("/Users/mac/Desktop/CMU/10701MachineLearning/project/10701-mvf-bto/pre_train/midterm"):
    print(filename,type(filename))

Dh6p3ID40 <class 'str'>
Dh6p3ID30 <class 'str'>
Dh6p1ID20 <class 'str'>
Dh4p3ID20 <class 'str'>
Dh6p2ID20 <class 'str'>
Dh6p3ID20 <class 'str'>
Dh3p3ID20 <class 'str'>
Dh6p3ID20e20 <class 'str'>
Dh6p3ID20e10 <class 'str'>


In [6]:
import tqdm
models=[]
for filename in tqdm.tqdm(os.listdir("/Users/mac/Desktop/CMU/10701MachineLearning/project/10701-mvf-bto/pre_train/midterm")):
    model_name="/Users/mac/Desktop/CMU/10701MachineLearning/project/10701-mvf-bto/pre_train/midterm/"+filename
    # print(filename)
    model = load_model(model_name)
    models.append(model)

  0%|          | 0/9 [00:00<?, ?it/s]2022-10-30 09:32:40.597055: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
100%|██████████| 9/9 [02:08<00:00, 14.31s/it]


## Preprocessing to create model inputs and targets

In [7]:
train_split = 0.7
test_split = 0.2
# by default uses validation_split = 1 - (train_split + test_split)


In [8]:
(datasets, 
train_cells, 
test_cells, 
validation_cells) = create_discharge_inputs(data, train_split, test_split, forecast_horizon=3, history_window=6)

 Data for cell b1c3 is corrupted. Skipping cell.
 Data for cell b1c8 is corrupted. Skipping cell.


100%|██████████| 905/905 [00:08<00:00, 105.29it/s]


In [9]:
print(train_cells, test_cells, validation_cells)
window_length = datasets["X_train"].shape[1]
n_features = datasets["X_train"].shape[2]
batch_input_shape = (datasets["batch_size"], window_length, n_features)
n_outputs = datasets["y_train"].shape[-1]
nf_steps = datasets["y_train"].shape[1]
y = datasets["y_train"][:, 0, 0]
idx = y < 2.9
weights = np.ones_like(y) * 1
weights[idx] = 2
print(window_length, n_features, batch_input_shape, n_outputs, nf_steps)

['b1c0', 'b1c1', 'b1c2', 'b1c4', 'b1c5', 'b1c6'] ['b1c7'] ['b1c9', 'b1c10']
5 4 (11, 5, 4) 2 3


## Train Model

In [10]:
# window_length = datasets["X_train"].shape[1]
# n_features = datasets["X_train"].shape[2]
# batch_input_shape = (datasets["batch_size"], window_length, n_features)
# n_outputs = datasets["y_train"].shape[-1]
# nf_steps = datasets["y_train"].shape[1]
# y = datasets["y_train"][:, 0, 0]
# idx = y < 2.9
# weights = np.ones_like(y) * 1
# weights[idx] = 2
# print(window_length, n_features, batch_input_shape, n_outputs, nf_steps)

In [11]:
# model = BaselineLSTM(batch_input_shape=batch_input_shape, n_outputs=n_outputs, nf_steps=nf_steps)

In [12]:
# model.compile(optimizer="adam", loss="mse", metrics=[MeanSquaredError()])

# es = EarlyStopping(
#     monitor="val_mean_squared_error",
#     min_delta=0,
#     patience=10,
#     verbose=1,
#     mode="auto",
#     restore_best_weights=True,
# )

# history = model.fit(
#     datasets["X_train"],
#     datasets["y_train"],
#     validation_data=(datasets["X_val"], datasets["y_val"]),
#     epochs=250,
#     batch_size=datasets["batch_size"],
#     shuffle=False,
#     callbacks=[es],
#     verbose=1,
#     sample_weight=weights

# )

In [13]:
# fig = go.Figure()
# fig.add_trace(
#     go.Scatter(
#         x=np.linspace(1, 50),
#         y=history.history["loss"],
#         showlegend=False,
#         mode="markers+lines",
#     )
# )
# fig.update_xaxes(title="Epochs")
# fig.update_yaxes(title="Loss (MSE)")

## Parity Plot of baseline (Dh6p3ID20) Training Error

In [14]:
# random plotting traing error at some interval = skip to not make the plot rendering too slow

batch_size = datasets["batch_size"]
skip = 70

fig = go.Figure()
fig.add_trace(go.Scatter(x=[0, 1], y=[0, 1], showlegend=False, mode="markers+lines"))
for i in range(0, len(datasets["X_train"]), batch_size * skip):
    df_pred = pd.DataFrame(models[5].predict(datasets["X_train"][i : i + batch_size], verbose=0)[:, :, 0])
    df_train = pd.DataFrame(datasets["y_train"][i : i + batch_size][:, :, 0])
    fig.add_trace(
        go.Scatter(
            x=df_pred[0].values,
            y=df_train[0].values,
            showlegend=False,
            mode="markers+lines",
        )
    )

fig.update_yaxes(title="Normalized Voltage Target")
fig.update_xaxes(title="Normalized Voltage Prediction")
fig.update_layout(template="simple_white")

In [15]:
fig = go.Figure()
fig.add_trace(go.Scatter(x=[0, 1], y=[0, 1], showlegend=False, mode="markers+lines"))
for i in range(0, len(datasets["X_train"]), batch_size * skip):
    df_pred = pd.DataFrame(models[5].predict(datasets["X_train"][i : i + batch_size], verbose=0)[:, :, 0])
    df_train = pd.DataFrame(datasets["y_train"][i : i + batch_size][:, :, 0])
    fig.add_trace(
        go.Scatter(
            x=df_pred[0].values,
            y=df_train[0].values,
            showlegend=False,
            mode="markers+lines",
        )
    )

fig.update_yaxes(title="Normalized Temperature Target")
fig.update_xaxes(title="Normalized Temperature Prediction")
fig.update_layout(template="simple_white")

## Parity Plot of baseline (Dh6p3ID20) Test Error

In [16]:
skip = 20

fig = go.Figure()
fig.add_trace(go.Scatter(x=[0, 1], y=[0, 1], showlegend=False, mode="markers+lines"))
for i in range(0, len(datasets["X_test"]), batch_size * skip):
    df_pred = pd.DataFrame(models[5].predict(datasets["X_test"][i : i + batch_size], verbose=0)[:, :, 0])
    df_train = pd.DataFrame(datasets["y_test"][i : i + batch_size][:, :, 0])
    fig.add_trace(
        go.Scatter(
            x=df_pred[0].values,
            y=df_train[0].values,
            showlegend=False,
            mode="markers+lines",
        )
    )

fig.update_yaxes(title="Normalized Voltage Target")
fig.update_xaxes(title="Normalized Voltage Prediction")
fig.update_layout(template="simple_white")

In [17]:
fig = go.Figure()
fig.add_trace(go.Scatter(x=[0, 1], y=[0, 1], showlegend=False, mode="markers+lines"))
for i in range(0, len(datasets["X_test"]), batch_size * skip):
    df_pred = pd.DataFrame(models[5].predict(datasets["X_test"][i : i + batch_size], verbose=0)[:, :, 1])
    df_train = pd.DataFrame(datasets["y_test"][i : i + batch_size][:, :, 1])
    fig.add_trace(
        go.Scatter(
            x=df_pred[0].values,
            y=df_train[0].values,
            showlegend=False,
            mode="markers+lines",
        )
    )

fig.update_yaxes(title="Normalized Temperature Target")
fig.update_xaxes(title="Normalized Temperature Prediction")
fig.update_layout(template="simple_white")

## True vs Predicted Traces (Train Set) (Dh6p3ID20)

In [18]:
train_cell_ids = datasets['original_train']['Cell'].unique()
train_cell_ids

array(['b1c0', 'b1c1', 'b1c2', 'b1c4', 'b1c5', 'b1c6'], dtype=object)

In [19]:
symbol_list = ["circle-open", "circle", "triangle-up"]
pallete = plotly.colors.qualitative.Dark24 + plotly.colors.qualitative.T10
pallete = pallete*70000

In [21]:
skip = 500
train_cell_id_idx = 0
last_cycle = 1
current_cycle = 1
opacity_list = [1, 0.6, 0.3]
fig = go.Figure()
for i in range(0, len(datasets["X_train"]), batch_size * skip):
    df_true = pd.DataFrame(datasets["y_train"][i : i + batch_size][:, 0, 0])
    
    last_cycle = current_cycle
    current_cycle = int(datasets["X_train"][i : i + batch_size][0][0][-1]*MAX_CYCLE)
    if current_cycle>740:
        continue
    if last_cycle> current_cycle:
        train_cell_id_idx += 1
        
    cell_id = train_cell_ids[train_cell_id_idx]
    original_df = datasets["original_train"]

    original_df= original_df[original_df.Cycle==current_cycle]
    original_df= original_df[original_df.Cell==cell_id]
    original_df = original_df[original_df.I < MAX_DISCHARGE_CURRENT]
    original_df = original_df[original_df.I > MIN_DISCHARGE_CURRENT]

    original_df['Qd'] = (original_df['Qd']-original_df['Qd'].min())/(original_df['Qd'].max()-original_df['Qd'].min())

    if len(original_df)< 2:
        continue
    q_new = original_df.Qd.values
    t_new = original_df['t'].values
    time_interpolator = interp1d(x = q_new,  y = t_new, fill_value="extrapolate")
    
    
    for j in range(nf_steps):
        df_pred = pd.DataFrame(models[5].predict(datasets["X_train"][i : i + batch_size], verbose=0)[:, j, 0])
    
        ref_capacities_wrt_nf = REFERENCE_CAPACITIES[window_length+j:-nf_steps+j-1]
        prediction_interpolator = interp1d(x = ref_capacities_wrt_nf, 
                                           y = df_pred[0].values*(VOLTAGE_MAX - VOLTAGE_MIN) + VOLTAGE_MIN,
                                           
                                          )
        
        q_j = q_new[q_new>= min(REFERENCE_CAPACITIES[window_length+j:-nf_steps+j-1])]
        q_j = q_j[q_j<= max(REFERENCE_CAPACITIES[window_length+j:-nf_steps+j-1])]
        t_interp = time_interpolator(REFERENCE_CAPACITIES[window_length:-nf_steps-1])
        if j==0:
            q_j0 = q_j
        V_pred = prediction_interpolator(q_j)
        fig.add_trace(
            go.Scatter(
                x=time_interpolator(q_j),
                y=V_pred,
                showlegend=True,
                mode="lines",
                line_dash="dash",
                name = "Interpolated Predictions",
                marker_color=pallete[i],
                opacity=opacity_list[j]
            )
        )
        
        fig.add_trace(
            go.Scatter(
                x=time_interpolator(ref_capacities_wrt_nf),
                y=df_pred[0].values*(VOLTAGE_MAX - VOLTAGE_MIN) + VOLTAGE_MIN,
                showlegend=True,
                mode="markers",
                name = f"Predictions Forecast Horizon {j+1}",
                marker_color=pallete[i],
                marker_symbol = symbol_list[j],
                marker_size=8,
#                 marker_opacity=opacity_list[j]
            )
        )


    odf = original_df[original_df.Qd > q_j0.min()]
    odf = odf[odf.Qd < q_j.max()]
    fig.add_trace(
        go.Scatter(
            x=odf['t'],
            y=odf['V'],
            showlegend=True,
            mode="lines",
            name = f"{cell_id} Cycle {current_cycle}",
            line_color=pallete[i]
        )
    )


fig.update_yaxes(title="Voltage [V]", showgrid=True)
fig.update_xaxes(title="Times (min)", showgrid=True)
fig.update_layout(template="simple_white")

In [23]:
train_cell_id_idx = 0
last_cycle = 1
current_cycle = 1

fig = go.Figure()
for i in range(0, len(datasets["X_train"]), batch_size * skip):
    df_true = pd.DataFrame(datasets["y_train"][i : i + batch_size][:, 0, 1])
    
    last_cycle = current_cycle
    current_cycle = int(datasets["X_train"][i : i + batch_size][0][0][-1]*MAX_CYCLE)
    if current_cycle>740:
        continue
    if last_cycle> current_cycle:
        train_cell_id_idx += 1
        
    cell_id = train_cell_ids[train_cell_id_idx]
    original_df = datasets["original_train"]

    original_df= original_df[original_df.Cycle==current_cycle]
    original_df= original_df[original_df.Cell==cell_id]
    original_df = original_df[original_df.I < MAX_DISCHARGE_CURRENT]
    original_df = original_df[original_df.I > MIN_DISCHARGE_CURRENT]

    original_df['Qd'] = (original_df['Qd']-original_df['Qd'].min())/(original_df['Qd'].max()-original_df['Qd'].min())

    if len(original_df)< 2:
        continue
    q_new = original_df.Qd.values
    t_new = original_df['t'].values
    time_interpolator = interp1d(x = q_new,  y = t_new, fill_value="extrapolate")
    
    
    for j in range(nf_steps):
        df_pred = pd.DataFrame(models[5].predict(datasets["X_train"][i : i + batch_size], verbose=0)[:, j, 1])
    
        ref_capacities_wrt_nf = REFERENCE_CAPACITIES[window_length+j:-nf_steps+j-1]

        prediction_interpolator = interp1d(x = ref_capacities_wrt_nf, 
                                           y = df_pred[0].values*(TEMPERATURE_MAX - TEMPERATURE_MIN) + TEMPERATURE_MIN,
                                           
                                          )
        
        q_j = q_new[q_new>= min(REFERENCE_CAPACITIES[window_length+j:-nf_steps+j-1])]
        q_j = q_j[q_j<= max(REFERENCE_CAPACITIES[window_length+j:-nf_steps+j-1])]
        t_interp = time_interpolator(REFERENCE_CAPACITIES[window_length:-nf_steps-1])
        if j==0:
            q_j0 = q_j
        V_pred = prediction_interpolator(q_j)
        fig.add_trace(
            go.Scatter(
                x=time_interpolator(q_j),
                y=V_pred,
                showlegend=True,
                mode="lines",
                line_dash="dash",
                name = "Interpolated Predictions",
                marker_color=pallete[i//skip],
                opacity=opacity_list[j]
            )
        )
        
        fig.add_trace(
            go.Scatter(
                x=time_interpolator(ref_capacities_wrt_nf),
                y=df_pred[0].values*(TEMPERATURE_MAX - TEMPERATURE_MIN) + TEMPERATURE_MIN,
                showlegend=True,
                mode="markers",
                name = f"Predictions Forecast Horizon {j+1}",
                marker_color=pallete[i//skip],
                marker_symbol = symbol_list[j],
                marker_size=8,
            )
        )


    odf = original_df[original_df.Qd > q_j0.min()]
    odf = odf[odf.Qd < q_j.max()]
    fig.add_trace(
        go.Scatter(
            x=odf['t'],
            y=odf['temp'],
            showlegend=True,
            mode="lines",
            name = f"{cell_id} Cycle {current_cycle}",
            line_color=pallete[i//skip]
        )
    )


fig.update_yaxes(title="Temperature [°C]", showgrid=True)
fig.update_xaxes(title="Times (min)", showgrid=True)
fig.update_layout(template="simple_white")

## True vs Predicted Traces (Test Set) (Dh6p3ID20)

In [24]:
test_cell_ids = datasets['original_test']['Cell'].unique()
test_cell_ids
len(datasets["X_test"])

9548

In [25]:
skip = 300
test_cell_id_idx = 0
last_cycle = 1
current_cycle = 1
opacity_list = [1, 0.6, 0.3]
fig = go.Figure()
for i in range(0, len(datasets["X_test"]), batch_size * skip):
    df_true = pd.DataFrame(datasets["y_test"][i : i + batch_size][:, 0, 0])
    
    last_cycle = current_cycle
    current_cycle = int(datasets["X_test"][i : i + batch_size][0][0][-1]*MAX_CYCLE)

    if last_cycle> current_cycle:
        test_cell_id_idx += 1
        
    cell_id = test_cell_ids[test_cell_id_idx]
    original_df = datasets["original_test"]

    original_df= original_df[original_df.Cycle==current_cycle]
    original_df= original_df[original_df.Cell==cell_id]
    original_df = original_df[original_df.I < MAX_DISCHARGE_CURRENT]
    original_df = original_df[original_df.I > MIN_DISCHARGE_CURRENT]

    original_df['Qd'] = (original_df['Qd']-original_df['Qd'].min())/(original_df['Qd'].max()-original_df['Qd'].min())

    if len(original_df)< 2:
        continue
    q_new = original_df.Qd.values
    t_new = original_df['t'].values
    time_interpolator = interp1d(x = q_new,  y = t_new, fill_value="extrapolate")
    
    
    for j in range(nf_steps):
        df_pred = pd.DataFrame(models[5].predict(datasets["X_test"][i : i + batch_size], verbose=0)[:, j, 0])
    
        ref_capacities_wrt_nf = REFERENCE_CAPACITIES[window_length+j:-nf_steps+j-1]

        prediction_interpolator = interp1d(x = ref_capacities_wrt_nf, 
                                           y = df_pred[0].values*(VOLTAGE_MAX - VOLTAGE_MIN) + VOLTAGE_MIN,
                                           
                                          )
        
        q_j = q_new[q_new>= min(REFERENCE_CAPACITIES[window_length+j:-nf_steps+j-1])]
        q_j = q_j[q_j<= max(REFERENCE_CAPACITIES[window_length+j:-nf_steps+j-1])]
        t_interp = time_interpolator(REFERENCE_CAPACITIES[window_length:-nf_steps-1])
        if j==0:
            q_j0 = q_j
        V_pred = prediction_interpolator(q_j)
        fig.add_trace(
            go.Scatter(
                x=time_interpolator(q_j),
                y=V_pred,
                showlegend=True,
                mode="lines",
                line_dash="dash",
                name = "Interpolated Predictions",
                marker_color=pallete[i//skip],
                opacity=opacity_list[j]
            )
        )
        
        fig.add_trace(
            go.Scatter(
                x=time_interpolator(ref_capacities_wrt_nf),
                y=df_pred[0].values*(VOLTAGE_MAX - VOLTAGE_MIN) + VOLTAGE_MIN,
                showlegend=True,
                mode="markers",
                name = f"Predictions Forecast Horizon {j+1}",
                marker_color=pallete[i//skip],
                marker_symbol = symbol_list[j],
                marker_size=8,
#                 marker_opacity=opacity_list[j]
            )
        )


    odf = original_df[original_df.Qd > q_j0.min()]
    odf = odf[odf.Qd < q_j.max()]
    fig.add_trace(
        go.Scatter(
            x=odf['t'],
            y=odf['V'],
            showlegend=True,
            mode="lines",
            name = f"{cell_id} Cycle {current_cycle}",
            line_color=pallete[i//skip]
        )
    )


fig.update_yaxes(title="Voltage [V]", showgrid=True)
fig.update_xaxes(title="Times (min)", showgrid=True)
fig.update_layout(template="simple_white")

In [28]:
test_cell_id_idx = 0
last_cycle = 1
current_cycle = 1

fig = go.Figure()
for i in range(0, len(datasets["X_test"]), batch_size * skip):
    
    last_cycle = current_cycle
    current_cycle = int(datasets["X_test"][i : i + batch_size][0][0][-1]*MAX_CYCLE)
    
    if last_cycle> current_cycle:
        test_cell_id_idx += 1
        
    cell_id = test_cell_ids[test_cell_id_idx]
    original_df = datasets["original_test"]

    original_df= original_df[original_df.Cycle==current_cycle]
    original_df= original_df[original_df.Cell==cell_id]
    original_df = original_df[original_df.I < MAX_DISCHARGE_CURRENT]
    original_df = original_df[original_df.I > MIN_DISCHARGE_CURRENT]

    original_df['Qd'] = (original_df['Qd']-original_df['Qd'].min())/(original_df['Qd'].max()-original_df['Qd'].min())

    if len(original_df)< 2:
        continue
    q_new = original_df.Qd.values
    t_new = original_df['t'].values
    time_interpolator = interp1d(x = q_new,  y = t_new, fill_value="extrapolate")
    
    
    for j in range(nf_steps):
        df_pred = pd.DataFrame(models[5].predict(datasets["X_test"][i : i + batch_size], verbose=0)[:, j, 1])
    
        ref_capacities_wrt_nf = REFERENCE_CAPACITIES[window_length+j:-nf_steps+j-1]

        prediction_interpolator = interp1d(x = ref_capacities_wrt_nf, 
                                           y = df_pred[0].values*(TEMPERATURE_MAX - TEMPERATURE_MIN) + TEMPERATURE_MIN,
                                           
                                          )
        
        q_j = q_new[q_new>= min(REFERENCE_CAPACITIES[window_length+j:-nf_steps+j-1])]
        q_j = q_j[q_j<= max(REFERENCE_CAPACITIES[window_length+j:-nf_steps+j-1])]
        t_interp = time_interpolator(REFERENCE_CAPACITIES[window_length:-nf_steps-1])
        if j==0:
            q_j0 = q_j
        V_pred = prediction_interpolator(q_j)
        fig.add_trace(
            go.Scatter(
                x=time_interpolator(q_j),
                y=V_pred,
                showlegend=True,
                mode="lines",
                line_dash="dash",
                name = f"Interpolated Predictions {j+1}",
                marker_color=pallete[i//skip],
                opacity=opacity_list[j]
            )
        )
        
        fig.add_trace(
            go.Scatter(
                x=time_interpolator(ref_capacities_wrt_nf),
                y=df_pred[0].values*(TEMPERATURE_MAX - TEMPERATURE_MIN) + TEMPERATURE_MIN,
                showlegend=True,
                mode="markers",
                name = f"Predictions Forecast Horizon {j+1}",
                marker_color=pallete[i//skip],
                marker_symbol = symbol_list[j],
                marker_size=8,
#                 marker_opacity=opacity_list[j]
            )
        )


    odf = original_df[original_df.Qd > q_j0.min()]
    odf = odf[odf.Qd < q_j.max()]
    fig.add_trace(
        go.Scatter(
            x=odf['t'],
            y=odf['temp'],
            showlegend=True,
            mode="lines",
            name = f"{cell_id} Cycle {current_cycle}",
            line_color=pallete[i//skip]
        )
    )


fig.update_yaxes(title="Temperature [°C]", showgrid=True)
fig.update_xaxes(title="Times (min)", showgrid=True)
fig.update_layout(template="simple_white")

## Metrics (Dh6p3ID20)

In [29]:
def error_calculation(model, datasets, error_function):
    results = {}
    batch_size = datasets["batch_size"]
    n_outputs = datasets["y_train"].shape[-1]
    nf_steps = datasets["y_train"].shape[1]
    for dset in ["train", "test"]:
        skip = 200 if dset == "train"  else 50
        for output in range(n_outputs):
            for step in range(nf_steps):
                collector = []
                for i in range(0, len(datasets[f"X_{dset}"]), batch_size*skip):
                    true = datasets[f"y_{dset}"][i : i + batch_size][:, j, output]
                    pred = model.predict(datasets[f"X_{dset}"][i : i + batch_size], verbose=0)[:, j, output]
                    error = error_function(true, pred)
                    collector.append(error)
                results[f"{dset}_output{output}_forecasthorizon{step}"] =sum(collector)/len(collector)
                         
    return results

In [30]:
root_mean_square_error = lambda y_true, y_pred : np.sqrt(((y_true - y_pred)**2).sum()/len(y_true))
mean_absolute_error = lambda y_true, y_pred : abs(y_true - y_pred).sum()/len(y_true)
mean_absolute_percent_error = lambda y_true, y_pred : (abs(y_true - y_pred)/y_true).sum()/len(y_true)

In [31]:
error_calculation(models[5], datasets, error_function=mean_absolute_error)

{'train_output0_forecasthorizon0': 0.027659873779097788,
 'train_output0_forecasthorizon1': 0.027560590774132105,
 'train_output0_forecasthorizon2': 0.027559906921668383,
 'train_output1_forecasthorizon0': 0.01840623482724739,
 'train_output1_forecasthorizon1': 0.01840624500765503,
 'train_output1_forecasthorizon2': 0.01840624500765503,
 'test_output0_forecasthorizon0': 0.042493311217554224,
 'test_output0_forecasthorizon1': 0.042184288829481296,
 'test_output0_forecasthorizon2': 0.04214548801193387,
 'test_output1_forecasthorizon0': 0.03488739367790689,
 'test_output1_forecasthorizon1': 0.03489960299701194,
 'test_output1_forecasthorizon2': 0.03490311365038249}

## Comparison between different forecast horizon Dh6p1ID20, Dh6p2ID20, Dh6p3ID20

In [33]:
# Dh6p3ID40 <class 'str'>
# Dh6p3ID30 <class 'str'>
# Dh6p1ID20 <class 'str'>
# Dh4p3ID20 <class 'str'>
# Dh6p2ID20 <class 'str'>
# Dh6p3ID20 <class 'str'>
# Dh3p3ID20 <class 'str'>
# Dh6p3ID20e20 <class 'str'>
# Dh6p3ID20e10 <class 'str'>

# [6, 6, 6, 4, 6, 6, 3, 6, 6]
# [3, 3, 1, 3, 2, 3, 3, 3, 3]

datasets_list=[]
history_window_list=[]
forecast_horizon_list=[]
for i in tqdm.tqdm(range(9)):
    if (i == 0 or i ==1 or i == 5 or i ==7 or i == 8):
        history_window=6
        forecast_horizon=3
    elif(i == 2):
        history_window=6
        forecast_horizon=1
    elif(i == 3):
        history_window=4
        forecast_horizon=3
    elif(i == 4):
        history_window=6
        forecast_horizon=2
    elif(i == 6):
        history_window=3
        forecast_horizon=3

    (datasets, 
    train_cells, 
    test_cells, 
    validation_cells) = create_discharge_inputs(data, train_split, test_split, forecast_horizon=forecast_horizon, history_window=history_window)

    datasets_list.append(datasets)
    history_window_list.append(history_window)
    forecast_horizon_list.append(forecast_horizon)

print(history_window_list)
print(forecast_horizon_list)

  0%|          | 0/9 [00:00<?, ?it/s]

 Data for cell b1c3 is corrupted. Skipping cell.
 Data for cell b1c8 is corrupted. Skipping cell.


 11%|█         | 1/9 [01:39<13:19, 100.00s/it]

 Data for cell b1c3 is corrupted. Skipping cell.
 Data for cell b1c8 is corrupted. Skipping cell.


 22%|██▏       | 2/9 [03:11<11:03, 94.77s/it] 

...

 Data for cell b1c3 is corrupted. Skipping cell.
 Data for cell b1c8 is corrupted. Skipping cell.


 89%|████████▉ | 8/9 [11:52<01:28, 88.32s/it]

 Data for cell b1c3 is corrupted. Skipping cell.
 Data for cell b1c8 is corrupted. Skipping cell.


100%|██████████| 9/9 [13:20<00:00, 88.96s/it]

[6, 6, 6, 4, 6, 6, 3, 6, 6]
[3, 3, 1, 3, 2, 3, 3, 3, 3]


In [113]:
# # Dh6p1ID20 -> 2, Dh6p2ID20 -> 4, Dh6p3ID20 -> 5
# voltage
list=[2,4,5]
fig = go.Figure()
fig.layout.height=500
fig.layout.width=500
fig.layout.legend.x=0
fig.layout.legend.y=1.0
for cell in list:
    if(cell == 2):
        name = 'Pred1'
    elif(cell == 4):
        name = "Pred2"
    else:
        name = "Pred3"
    window_length = datasets_list[cell]["X_train"].shape[1]
    n_features = datasets_list[cell]["X_train"].shape[2]
    batch_input_shape = (datasets_list[cell]["batch_size"], window_length, n_features)
    n_outputs = datasets_list[cell]["y_train"].shape[-1]
    nf_steps = datasets_list[cell]["y_train"].shape[1]
    y = datasets_list[cell]["y_train"][:, 0, 0]
    idx = y < 2.9
    weights = np.ones_like(y) * 1
    weights[idx] = 2
    batch_size = datasets_list[cell]["batch_size"]
    print(window_length, n_features, batch_input_shape, n_outputs, nf_steps)


    skip = 1000
    test_cell_id_idx = 0
    last_cycle = 1
    current_cycle = 1
    opacity_list = [1, 0.6, 0.3]
    
    for i in range(0, len(datasets_list[cell]["X_test"]), batch_size * skip):
        df_true = pd.DataFrame(datasets_list[cell]["y_test"][i : i + batch_size][:, 0, 0])
    
        last_cycle = current_cycle
        current_cycle = int(datasets_list[cell]["X_test"][i : i + batch_size][0][0][-1]*MAX_CYCLE)

        if last_cycle> current_cycle:
            test_cell_id_idx += 1
        
        cell_id = test_cell_ids[test_cell_id_idx]
        original_df = datasets_list[cell]["original_test"]

        original_df= original_df[original_df.Cycle==current_cycle]
        original_df= original_df[original_df.Cell==cell_id]
        original_df = original_df[original_df.I < MAX_DISCHARGE_CURRENT]
        original_df = original_df[original_df.I > MIN_DISCHARGE_CURRENT]

        original_df['Qd'] = (original_df['Qd']-original_df['Qd'].min())/(original_df['Qd'].max()-original_df['Qd'].min())

        if len(original_df)< 2:
            continue
        q_new = original_df.Qd.values
        t_new = original_df['t'].values
        time_interpolator = interp1d(x = q_new,  y = t_new, fill_value="extrapolate")
    
    
        for j in range(nf_steps):
            df_pred = pd.DataFrame(models[cell].predict(datasets_list[cell]["X_test"][i : i + batch_size], verbose=0)[:, j, 0])
    
            ref_capacities_wrt_nf = REFERENCE_CAPACITIES[window_length+j:-nf_steps+j-1]

            error_interpolator = interp1d(x = ref_capacities_wrt_nf, 
                                           y = abs(df_pred[0].values-df_true[0].values)*(VOLTAGE_MAX - VOLTAGE_MIN),
                                           
                                          )
        
            q_j = q_new[q_new>= min(REFERENCE_CAPACITIES[window_length+j:-nf_steps+j-1])]
            q_j = q_j[q_j<= max(REFERENCE_CAPACITIES[window_length+j:-nf_steps+j-1])]
            t_interp = time_interpolator(REFERENCE_CAPACITIES[window_length:-nf_steps-1])
            if j==0:
                q_j0 = q_j
            error_intp = error_interpolator(q_j)

            fig.add_trace(
                go.Scatter(
                    x=time_interpolator(q_j),
                    y=error_intp,
                    showlegend=False,
                    mode="lines",
                    line_dash="dash",
                    name = "Interpolated Predictions "+name,
                    marker_color=pallete[cell],
                    opacity=opacity_list[j]
                )
            )
        
            fig.add_trace(
                go.Scatter(
                    x=time_interpolator(ref_capacities_wrt_nf),
                    y=abs(df_pred[0].values-df_true[0].values)*(VOLTAGE_MAX - VOLTAGE_MIN),
                    showlegend=True,
                    mode="markers",
                    name = f"Predicted (Forecast Horizon={j+1})"+' '+name,
                    marker_color=pallete[cell],
                    marker_symbol = symbol_list[j],
                    marker_size=8,
#                 marker_opacity=opacity_list[j]
                )
            )


# odf = original_df[original_df.Qd > q_j0.min()]
# odf = odf[odf.Qd < q_j.max()]
# fig.add_trace(
# go.Scatter(
#             x=odf['t'],
#                 y=odf['V'],
#                 showlegend=True,
#                 mode="lines",
#                 name = f"{cell_id} Cycle {current_cycle} ",
#                 line_color=pallete[cell+3]
#             )
#         )


fig.update_yaxes(title="Voltage MAE [V]", showgrid=True)
fig.update_xaxes(title="Times (min)", showgrid=True)
fig.update_layout(template="simple_white")

5 4 (13, 5, 4) 2 1
5 4 (12, 5, 4) 2 2
5 4 (11, 5, 4) 2 3


In [112]:
# # Dh6p1ID20 -> 2, Dh6p2ID20 -> 4, Dh6p3ID20 -> 5
# temperature
list=[2,4,5]
fig = go.Figure()
fig.layout.height=620
fig.layout.width=620
fig.layout.legend.x=0
fig.layout.legend.y=1.0
for cell in list:
    if(cell == 2):
        name = 'Pred1'
    elif(cell == 4):
        name = "Pred2"
    else:
        name = "Pred3"
    window_length = datasets_list[cell]["X_train"].shape[1]
    n_features = datasets_list[cell]["X_train"].shape[2]
    batch_input_shape = (datasets_list[cell]["batch_size"], window_length, n_features)
    n_outputs = datasets_list[cell]["y_train"].shape[-1]
    nf_steps = datasets_list[cell]["y_train"].shape[1]
    y = datasets_list[cell]["y_train"][:, 0, 1]
    idx = y < 2.9
    weights = np.ones_like(y) * 1
    weights[idx] = 2
    batch_size = datasets_list[cell]["batch_size"]
    print(window_length, n_features, batch_input_shape, n_outputs, nf_steps)


    skip = 1000
    test_cell_id_idx = 0
    last_cycle = 1
    current_cycle = 1
    opacity_list = [1, 0.6, 0.3]
    
    for i in range(0, len(datasets_list[cell]["X_test"]), batch_size * skip):
        df_true = pd.DataFrame(datasets_list[cell]["y_test"][i : i + batch_size][:, 0, 1])
    
        last_cycle = current_cycle
        current_cycle = int(datasets_list[cell]["X_test"][i : i + batch_size][0][0][-1]*MAX_CYCLE)

        if last_cycle> current_cycle:
            test_cell_id_idx += 1
        
        cell_id = test_cell_ids[test_cell_id_idx]
        original_df = datasets_list[cell]["original_test"]

        original_df= original_df[original_df.Cycle==current_cycle]
        original_df= original_df[original_df.Cell==cell_id]
        original_df = original_df[original_df.I < MAX_DISCHARGE_CURRENT]
        original_df = original_df[original_df.I > MIN_DISCHARGE_CURRENT]

        original_df['Qd'] = (original_df['Qd']-original_df['Qd'].min())/(original_df['Qd'].max()-original_df['Qd'].min())

        if len(original_df)< 2:
            continue
        q_new = original_df.Qd.values
        t_new = original_df['t'].values
        time_interpolator = interp1d(x = q_new,  y = t_new, fill_value="extrapolate")
    
    
        for j in range(nf_steps):
            df_pred = pd.DataFrame(models[cell].predict(datasets_list[cell]["X_test"][i : i + batch_size], verbose=0)[:, j, 1])
    
            ref_capacities_wrt_nf = REFERENCE_CAPACITIES[window_length+j:-nf_steps+j-1]

            error_interpolator = interp1d(x = ref_capacities_wrt_nf, 
                                           y = abs(df_pred[0].values-df_true[0].values)*(TEMPERATURE_MAX - TEMPERATURE_MIN),
                                           
                                          )
        
            q_j = q_new[q_new>= min(REFERENCE_CAPACITIES[window_length+j:-nf_steps+j-1])]
            q_j = q_j[q_j<= max(REFERENCE_CAPACITIES[window_length+j:-nf_steps+j-1])]
            t_interp = time_interpolator(REFERENCE_CAPACITIES[window_length:-nf_steps-1])
            if j==0:
                q_j0 = q_j
            error_intp = error_interpolator(q_j)

            fig.add_trace(
                go.Scatter(
                    x=time_interpolator(q_j),
                    y=error_intp,
                    showlegend=False,
                    mode="lines",
                    line_dash="dash",
                    name = "Interpolated Predictions "+name,
                    marker_color=pallete[cell],
                    opacity=opacity_list[j]
                )
            )
        
            fig.add_trace(
                go.Scatter(
                    x=time_interpolator(ref_capacities_wrt_nf),
                    y=abs(df_pred[0].values-df_true[0].values)*(TEMPERATURE_MAX - TEMPERATURE_MIN),
                    showlegend=True,
                    mode="markers",
                    name = f"Predicted (Forecast Horizon={j+1})"+' '+name,
                    marker_color=pallete[cell],
                    marker_symbol = symbol_list[j],
                    marker_size=8,
#                 marker_opacity=opacity_list[j]
                )
            )


# odf = original_df[original_df.Qd > q_j0.min()]
# odf = odf[odf.Qd < q_j.max()]
# fig.add_trace(
# go.Scatter(
#             x=odf['t'],
#                 y=odf['V'],
#                 showlegend=True,
#                 mode="lines",
#                 name = f"{cell_id} Cycle {current_cycle} ",
#                 line_color=pallete[cell+3]
#             )
#         )


fig.update_yaxes(title="Temperature MAE [K]", showgrid=True)
fig.update_xaxes(title="Times (min)", showgrid=True)
fig.update_layout(template="simple_white")

5 4 (13, 5, 4) 2 1
5 4 (12, 5, 4) 2 2
5 4 (11, 5, 4) 2 3


## Comparison between different history size Dh6p3ID20, Dh4p3ID20, Dh3p3ID20

In [115]:
# Dh6p3ID20 -> 5, Dh4p3ID20 -> 3, Dh3p3ID20 -> 6
# voltage
list=[5,3,6]
fig = go.Figure()
fig.layout.height=500
fig.layout.width=500
fig.layout.legend.x=0
fig.layout.legend.y=1.0
for cell in list:
    if(cell == 5):
        name = 'Hist6'
    elif(cell == 3):
        name = "Hist4"
    else:
        name = "Hist3"
    window_length = datasets_list[cell]["X_train"].shape[1]
    n_features = datasets_list[cell]["X_train"].shape[2]
    batch_input_shape = (datasets_list[cell]["batch_size"], window_length, n_features)
    n_outputs = datasets_list[cell]["y_train"].shape[-1]
    nf_steps = datasets_list[cell]["y_train"].shape[1]
    y = datasets_list[cell]["y_train"][:, 0, 0]
    idx = y < 2.9
    weights = np.ones_like(y) * 1
    weights[idx] = 2
    batch_size = datasets_list[cell]["batch_size"]
    print(window_length, n_features, batch_input_shape, n_outputs, nf_steps)


    skip = 1000
    test_cell_id_idx = 0
    last_cycle = 1
    current_cycle = 1
    opacity_list = [1, 0.6, 0.3]
    
    for i in range(0, len(datasets_list[cell]["X_test"]), batch_size * skip):
        df_true = pd.DataFrame(datasets_list[cell]["y_test"][i : i + batch_size][:, 0, 0])
    
        last_cycle = current_cycle
        current_cycle = int(datasets_list[cell]["X_test"][i : i + batch_size][0][0][-1]*MAX_CYCLE)

        if last_cycle> current_cycle:
            test_cell_id_idx += 1
        
        cell_id = test_cell_ids[test_cell_id_idx]
        original_df = datasets_list[cell]["original_test"]

        original_df= original_df[original_df.Cycle==current_cycle]
        original_df= original_df[original_df.Cell==cell_id]
        original_df = original_df[original_df.I < MAX_DISCHARGE_CURRENT]
        original_df = original_df[original_df.I > MIN_DISCHARGE_CURRENT]

        original_df['Qd'] = (original_df['Qd']-original_df['Qd'].min())/(original_df['Qd'].max()-original_df['Qd'].min())

        if len(original_df)< 2:
            continue
        q_new = original_df.Qd.values
        t_new = original_df['t'].values
        time_interpolator = interp1d(x = q_new,  y = t_new, fill_value="extrapolate")
    
    
        for j in range(nf_steps):
            df_pred = pd.DataFrame(models[cell].predict(datasets_list[cell]["X_test"][i : i + batch_size], verbose=0)[:, j, 0])
    
            ref_capacities_wrt_nf = REFERENCE_CAPACITIES[window_length+j:-nf_steps+j-1]

            error_interpolator = interp1d(x = ref_capacities_wrt_nf, 
                                           y = abs(df_pred[0].values-df_true[0].values)*(VOLTAGE_MAX - VOLTAGE_MIN),
                                           
                                          )
        
            q_j = q_new[q_new>= min(REFERENCE_CAPACITIES[window_length+j:-nf_steps+j-1])]
            q_j = q_j[q_j<= max(REFERENCE_CAPACITIES[window_length+j:-nf_steps+j-1])]
            t_interp = time_interpolator(REFERENCE_CAPACITIES[window_length:-nf_steps-1])
            if j==0:
                q_j0 = q_j
            error_intp = error_interpolator(q_j)

            fig.add_trace(
                go.Scatter(
                    x=time_interpolator(q_j),
                    y=error_intp,
                    showlegend=False,
                    mode="lines",
                    line_dash="dash",
                    name = "Interpolated Predictions "+name,
                    marker_color=pallete[cell],
                    opacity=opacity_list[j]
                )
            )
        
            fig.add_trace(
                go.Scatter(
                    x=time_interpolator(ref_capacities_wrt_nf),
                    y=abs(df_pred[0].values-df_true[0].values)*(VOLTAGE_MAX - VOLTAGE_MIN),
                    showlegend=True,
                    mode="markers",
                    name = f"Predicted (Forecast Horizon={j+1})"+' '+name,
                    marker_color=pallete[cell],
                    marker_symbol = symbol_list[j],
                    marker_size=8,
#                 marker_opacity=opacity_list[j]
                )
            )


# odf = original_df[original_df.Qd > q_j0.min()]
# odf = odf[odf.Qd < q_j.max()]
# fig.add_trace(
# go.Scatter(
#             x=odf['t'],
#                 y=odf['V'],
#                 showlegend=True,
#                 mode="lines",
#                 name = f"{cell_id} Cycle {current_cycle} ",
#                 line_color=pallete[cell+3]
#             )
#         )


fig.update_yaxes(title="Voltage MAE [V]", showgrid=True)
fig.update_xaxes(title="Times (min)", showgrid=True)
fig.update_layout(template="simple_white")

5 4 (11, 5, 4) 2 3
3 4 (13, 3, 4) 2 3
2 4 (14, 2, 4) 2 3


In [122]:
# Dh6p3ID20 -> 5, Dh4p3ID20 -> 3, Dh3p3ID20 -> 6
# voltage
list=[5,3,6]
fig = go.Figure()
fig.layout.height=650
fig.layout.width=650
fig.layout.legend.x=0
fig.layout.legend.y=1.0
for cell in list:
    if(cell == 5):
        name = 'H6'
    elif(cell == 3):
        name = "H4"
    else:
        name = "H3"
    window_length = datasets_list[cell]["X_train"].shape[1]
    n_features = datasets_list[cell]["X_train"].shape[2]
    batch_input_shape = (datasets_list[cell]["batch_size"], window_length, n_features)
    n_outputs = datasets_list[cell]["y_train"].shape[-1]
    nf_steps = datasets_list[cell]["y_train"].shape[1]
    y = datasets_list[cell]["y_train"][:, 0, 0]
    idx = y < 2.9
    weights = np.ones_like(y) * 1
    weights[idx] = 2
    batch_size = datasets_list[cell]["batch_size"]
    print(window_length, n_features, batch_input_shape, n_outputs, nf_steps)


    skip = 1000
    test_cell_id_idx = 0
    last_cycle = 1
    current_cycle = 1
    opacity_list = [1, 0.6, 0.3]
    
    for i in range(0, len(datasets_list[cell]["X_test"]), batch_size * skip):
        df_true = pd.DataFrame(datasets_list[cell]["y_test"][i : i + batch_size][:, 0, 1])
    
        last_cycle = current_cycle
        current_cycle = int(datasets_list[cell]["X_test"][i : i + batch_size][0][0][-1]*MAX_CYCLE)

        if last_cycle> current_cycle:
            test_cell_id_idx += 1
        
        cell_id = test_cell_ids[test_cell_id_idx]
        original_df = datasets_list[cell]["original_test"]

        original_df= original_df[original_df.Cycle==current_cycle]
        original_df= original_df[original_df.Cell==cell_id]
        original_df = original_df[original_df.I < MAX_DISCHARGE_CURRENT]
        original_df = original_df[original_df.I > MIN_DISCHARGE_CURRENT]

        original_df['Qd'] = (original_df['Qd']-original_df['Qd'].min())/(original_df['Qd'].max()-original_df['Qd'].min())

        if len(original_df)< 2:
            continue
        q_new = original_df.Qd.values
        t_new = original_df['t'].values
        time_interpolator = interp1d(x = q_new,  y = t_new, fill_value="extrapolate")
    
    
        for j in range(nf_steps):
            df_pred = pd.DataFrame(models[cell].predict(datasets_list[cell]["X_test"][i : i + batch_size], verbose=0)[:, j, 1])
    
            ref_capacities_wrt_nf = REFERENCE_CAPACITIES[window_length+j:-nf_steps+j-1]

            error_interpolator = interp1d(x = ref_capacities_wrt_nf, 
                                           y = abs(df_pred[0].values-df_true[0].values)*(TEMPERATURE_MAX - TEMPERATURE_MIN),
                                           
                                          )
        
            q_j = q_new[q_new>= min(REFERENCE_CAPACITIES[window_length+j:-nf_steps+j-1])]
            q_j = q_j[q_j<= max(REFERENCE_CAPACITIES[window_length+j:-nf_steps+j-1])]
            t_interp = time_interpolator(REFERENCE_CAPACITIES[window_length:-nf_steps-1])
            if j==0:
                q_j0 = q_j
            error_intp = error_interpolator(q_j)

            fig.add_trace(
                go.Scatter(
                    x=time_interpolator(q_j),
                    y=error_intp,
                    showlegend=False,
                    mode="lines",
                    line_dash="dash",
                    name = "Interpolated Predictions "+name,
                    marker_color=pallete[cell],
                    opacity=opacity_list[j]
                )
            )
        
            fig.add_trace(
                go.Scatter(
                    x=time_interpolator(ref_capacities_wrt_nf),
                    y=abs(df_pred[0].values-df_true[0].values)*(TEMPERATURE_MAX - TEMPERATURE_MIN),
                    showlegend=True,
                    mode="markers",
                    name = f"Predicted (FH={j+1})"+' '+name,
                    marker_color=pallete[cell],
                    marker_symbol = symbol_list[j],
                    marker_size=8,
#                 marker_opacity=opacity_list[j]
                )
            )


# odf = original_df[original_df.Qd > q_j0.min()]
# odf = odf[odf.Qd < q_j.max()]
# fig.add_trace(
# go.Scatter(
#             x=odf['t'],
#                 y=odf['V'],
#                 showlegend=True,
#                 mode="lines",
#                 name = f"{cell_id} Cycle {current_cycle} ",
#                 line_color=pallete[cell+3]
#             )
#         )


fig.update_yaxes(title="Temperature MAE [K]", showgrid=True)
fig.update_xaxes(title="Times (min)", showgrid=True)
fig.update_layout(template="simple_white")

5 4 (11, 5, 4) 2 3
3 4 (13, 3, 4) 2 3
2 4 (14, 2, 4) 2 3


In [138]:
# Interpolation points 30
print(REFERENCE_CAPACITIES2)
(datasets, 
    train_cells, 
    test_cells, 
    validation_cells) = create_discharge_inputs(data, train_split, test_split, forecast_horizon=3, history_window=6, q_eval= REFERENCE_CAPACITIES2)
datasets_list[1]=datasets
print(datasets["batch_size"])

[0.         0.03333333 0.06666667 0.1        0.13333333 0.16666667
 0.2        0.23333333 0.26666667 0.3        0.33333333 0.36666667
 0.4        0.43333333 0.46666667 0.5        0.53333333 0.56666667
 0.6        0.63333333 0.66666667 0.7        0.73333333 0.76666667
 0.8        0.83333333 0.86666667 0.9        0.93333333 0.96666667
 1.        ]
 Data for cell b1c3 is corrupted. Skipping cell.
 Data for cell b1c8 is corrupted. Skipping cell.


100%|██████████| 905/905 [00:06<00:00, 137.51it/s]


22


In [140]:
# Interpolation points 40
from constants import *
print(REFERENCE_CAPACITIES3)
(datasets, 
    train_cells, 
    test_cells, 
    validation_cells) = create_discharge_inputs(data, train_split, test_split, forecast_horizon=3, history_window=6,q_eval=REFERENCE_CAPACITIES3)
datasets_list[0]=datasets
print(datasets["batch_size"])

[0.    0.025 0.05  0.075 0.1   0.125 0.15  0.175 0.2   0.225 0.25  0.275
 0.3   0.325 0.35  0.375 0.4   0.425 0.45  0.475 0.5   0.525 0.55  0.575
 0.6   0.625 0.65  0.675 0.7   0.725 0.75  0.775 0.8   0.825 0.85  0.875
 0.9   0.925 0.95  0.975 1.   ]
 Data for cell b1c3 is corrupted. Skipping cell.
 Data for cell b1c8 is corrupted. Skipping cell.


100%|██████████| 905/905 [00:07<00:00, 122.16it/s]


32


## Comparison between different sequence length (interpolation) Dh6p3ID20, Dh4p3ID30, Dh3p3ID40

In [154]:
# Dh6p3ID20 -> 5, Dh6p3ID30 -> 1, Dh6p3ID40 -> 0
# voltage
list=[5,0]
fig = go.Figure()
fig.layout.height=500
fig.layout.width=500
fig.layout.legend.x=0
fig.layout.legend.y=1.0
REFERENCE_CAPACITIES_LIST=[np.linspace(0,1,41), np.linspace(0,1,31), REFERENCE_CAPACITIES,REFERENCE_CAPACITIES,REFERENCE_CAPACITIES,REFERENCE_CAPACITIES]
for cell in list:
    cell=int(cell)
    if(cell == 5):
        name = 'ID20'
    elif(cell == 1):
        name = "ID30"
    else:
        name = "ID40"
    window_length = datasets_list[cell]["X_train"].shape[1]
    n_features = datasets_list[cell]["X_train"].shape[2]
    batch_input_shape = (datasets_list[cell]["batch_size"], window_length, n_features)
    n_outputs = datasets_list[cell]["y_train"].shape[-1]
    nf_steps = datasets_list[cell]["y_train"].shape[1]
    y = datasets_list[cell]["y_train"][:, 0, 0]
    idx = y < 2.9
    weights = np.ones_like(y) * 1
    weights[idx] = 2
    batch_size = datasets_list[cell]["batch_size"]
    print(window_length, n_features, batch_input_shape, n_outputs, nf_steps)


    skip = 1000
    test_cell_id_idx = 0
    last_cycle = 1
    current_cycle = 1
    opacity_list = [1, 0.6, 0.3]
    
    for i in range(0, len(datasets_list[cell]["X_test"]), batch_size * skip):
        df_true = pd.DataFrame(datasets_list[cell]["y_test"][i : i + batch_size][:, 0, 0])
    
        last_cycle = current_cycle
        current_cycle = int(datasets_list[cell]["X_test"][i : i + batch_size][0][0][-1]*MAX_CYCLE)

        if last_cycle> current_cycle:
            test_cell_id_idx += 1
        
        cell_id = test_cell_ids[test_cell_id_idx]
        original_df = datasets_list[cell]["original_test"]

        original_df= original_df[original_df.Cycle==current_cycle]
        original_df= original_df[original_df.Cell==cell_id]
        original_df = original_df[original_df.I < MAX_DISCHARGE_CURRENT]
        original_df = original_df[original_df.I > MIN_DISCHARGE_CURRENT]

        original_df['Qd'] = (original_df['Qd']-original_df['Qd'].min())/(original_df['Qd'].max()-original_df['Qd'].min())

        if len(original_df)< 2:
            continue
        q_new = original_df.Qd.values
        t_new = original_df['t'].values
        time_interpolator = interp1d(x = q_new,  y = t_new, fill_value="extrapolate")
    
    
        for j in range(nf_steps):
            df_pred = pd.DataFrame(models[cell].predict(datasets_list[cell]["X_test"][i : i + batch_size], verbose=0)[:, j, 0])
    
            ref_capacities_wrt_nf = REFERENCE_CAPACITIES_LIST[cell][window_length+j:-nf_steps+j-1]

            error_interpolator = interp1d(x = ref_capacities_wrt_nf, 
                                        y = abs(df_pred[0].values-df_true[0].values)*(VOLTAGE_MAX - VOLTAGE_MIN),
                                        fill_value="extrapolate"   
                                          )
        
            q_j = q_new[q_new>= min(REFERENCE_CAPACITIES[window_length+j:-nf_steps+j-1])]
            q_j = q_j[q_j<= max(REFERENCE_CAPACITIES[window_length+j:-nf_steps+j-1])]
            t_interp = time_interpolator(REFERENCE_CAPACITIES[window_length:-nf_steps-1])
            if j==0:
                q_j0 = q_j
            error_intp = error_interpolator(q_j)

            fig.add_trace(
                go.Scatter(
                    x=time_interpolator(q_j),
                    y=error_intp,
                    showlegend=False,
                    mode="lines",
                    line_dash="dash",
                    name = "Interpolated Predictions "+name,
                    marker_color=pallete[cell],
                    opacity=opacity_list[j]
                )
            )
        
            fig.add_trace(
                go.Scatter(
                    x=time_interpolator(ref_capacities_wrt_nf),
                    y=abs(df_pred[0].values-df_true[0].values)*(VOLTAGE_MAX - VOLTAGE_MIN),
                    showlegend=True,
                    mode="markers",
                    name = f"Predicted (Forecast Horizon={j+1})"+' '+name,
                    marker_color=pallete[cell],
                    marker_symbol = symbol_list[j],
                    marker_size=8,
#                 marker_opacity=opacity_list[j]
                )
            )


# odf = original_df[original_df.Qd > q_j0.min()]
# odf = odf[odf.Qd < q_j.max()]
# fig.add_trace(
# go.Scatter(
#             x=odf['t'],
#                 y=odf['V'],
#                 showlegend=True,
#                 mode="lines",
#                 name = f"{cell_id} Cycle {current_cycle} ",
#                 line_color=pallete[cell+3]
#             )
#         )


fig.update_yaxes(title="Voltage MAE [V]", showgrid=True)
fig.update_xaxes(title="Times (min)", showgrid=True)
fig.update_layout(template="simple_white")

5 4 (11, 5, 4) 2 3
5 4 (32, 5, 4) 2 3


In [137]:
REFERENCE_CAPACITIES1 = [
    0.00,
    0.025,
    0.075,
    0.1,
    0.2,
    0.3,
    0.4,
    0.5,
    0.6,
    0.8,
    0.85,
    0.9,
    0.95,
    0.975,
    0.98,
    0.985,
    0.99,
    0.995,
    0.998,
    1.0,
]

REFERENCE_CAPACITIES2=np.linspace(0,1,31)
REFERENCE_CAPACITIES3=np.linspace(0,1,41)

In [160]:
list=[5,0]
fig = go.Figure()
fig.layout.height=500
fig.layout.width=1000
# fig.layout.legend.x=0
# fig.layout.legend.y=1.0
REFERENCE_CAPACITIES_LIST=[np.linspace(0,1,41), np.linspace(0,1,31), REFERENCE_CAPACITIES,REFERENCE_CAPACITIES,REFERENCE_CAPACITIES,REFERENCE_CAPACITIES]
for cell in list:
    cell=int(cell)
    if(cell == 5):
        name = 'ID20'
    elif(cell == 1):
        name = "ID30"
    else:
        name = "ID40"
    window_length = datasets_list[cell]["X_train"].shape[1]
    n_features = datasets_list[cell]["X_train"].shape[2]
    batch_input_shape = (datasets_list[cell]["batch_size"], window_length, n_features)
    n_outputs = datasets_list[cell]["y_train"].shape[-1]
    nf_steps = datasets_list[cell]["y_train"].shape[1]
    y = datasets_list[cell]["y_train"][:, 0, 0]
    idx = y < 2.9
    weights = np.ones_like(y) * 1
    weights[idx] = 2
    batch_size = datasets_list[cell]["batch_size"]
    print(window_length, n_features, batch_input_shape, n_outputs, nf_steps)


    skip = 200
    test_cell_id_idx = 0
    last_cycle = 1
    current_cycle = 1
    opacity_list = [1, 0.6, 0.3]
    
    for i in range(0, len(datasets_list[cell]["X_test"]), batch_size * skip):
        df_true = pd.DataFrame(datasets_list[cell]["y_test"][i : i + batch_size][:, 0, 0])
    
        last_cycle = current_cycle
        current_cycle = int(datasets_list[cell]["X_test"][i : i + batch_size][0][0][-1]*MAX_CYCLE)

        if last_cycle> current_cycle:
            test_cell_id_idx += 1
        
        cell_id = test_cell_ids[test_cell_id_idx]
        original_df = datasets_list[cell]["original_test"]

        original_df= original_df[original_df.Cycle==current_cycle]
        original_df= original_df[original_df.Cell==cell_id]
        original_df = original_df[original_df.I < MAX_DISCHARGE_CURRENT]
        original_df = original_df[original_df.I > MIN_DISCHARGE_CURRENT]

        original_df['Qd'] = (original_df['Qd']-original_df['Qd'].min())/(original_df['Qd'].max()-original_df['Qd'].min())

        if len(original_df)< 2:
            continue
        q_new = original_df.Qd.values
        t_new = original_df['t'].values
        time_interpolator = interp1d(x = q_new,  y = t_new, fill_value="extrapolate")
    
    
        for j in range(nf_steps):
            df_pred = pd.DataFrame(models[cell].predict(datasets_list[cell]["X_test"][i : i + batch_size], verbose=0)[:, j, 0])
    
            ref_capacities_wrt_nf = REFERENCE_CAPACITIES_LIST[cell][window_length+j:-nf_steps+j-1]

            error_interpolator = interp1d(x = ref_capacities_wrt_nf, 
                                        y = abs(df_pred[0].values)*(VOLTAGE_MAX - VOLTAGE_MIN)+VOLTAGE_MIN,
                                        fill_value="extrapolate"   
                                          )
        
            q_j = q_new[q_new>= min(REFERENCE_CAPACITIES[window_length+j:-nf_steps+j-1])]
            q_j = q_j[q_j<= max(REFERENCE_CAPACITIES[window_length+j:-nf_steps+j-1])]
            t_interp = time_interpolator(REFERENCE_CAPACITIES[window_length:-nf_steps-1])
            if j==0:
                q_j0 = q_j
            error_intp = error_interpolator(q_j)

            # fig.add_trace(
            #     go.Scatter(
            #         x=time_interpolator(q_j),
            #         y=error_intp,
            #         showlegend=False,
            #         mode="lines",
            #         line_dash="dash",
            #         name = "Interpolated Predictions "+name,
            #         marker_color=pallete[cell],
            #         opacity=opacity_list[j]
            #     )
            # )
        
            fig.add_trace(
                go.Scatter(
                    x=time_interpolator(ref_capacities_wrt_nf),
                    y=abs(df_pred[0].values)*(VOLTAGE_MAX - VOLTAGE_MIN)+VOLTAGE_MIN,
                    showlegend=True,
                    mode="markers",
                    name = f"Predicted (Forecast Horizon={j+1})"+' '+name,
                    marker_color=pallete[cell],
                    marker_symbol = symbol_list[j],
                    marker_size=8,
#                 marker_opacity=opacity_list[j]
                )
            )

            fig.add_trace(
                go.Scatter(
                    x=time_interpolator(ref_capacities_wrt_nf),
                    y=abs(df_true[0].values)*(VOLTAGE_MAX - VOLTAGE_MIN)+VOLTAGE_MIN,
                    showlegend=True,
                    mode="lines",
                    # name = f"Predicted (Forecast Horizon={j+1})"+' '+name,
                    marker_color=pallete[cell],
                    marker_symbol = symbol_list[j],
                    marker_size=8,
#                 marker_opacity=opacity_list[j]
                )
            )



# odf = original_df[original_df.Qd > q_j0.min()]
# odf = odf[odf.Qd < q_j.max()]
# fig.add_trace(
# go.Scatter(
#             x=odf['t'],
#                 y=odf['V'],
#                 showlegend=True,
#                 mode="lines",
#                 name = f"{cell_id} Cycle {current_cycle} ",
#                 line_color=pallete[cell+3]
#             )
#         )


fig.update_yaxes(title="Voltage MAE [V]", showgrid=True)
fig.update_xaxes(title="Times (min)", showgrid=True)
fig.update_layout(template="simple_white")

5 4 (11, 5, 4) 2 3
5 4 (32, 5, 4) 2 3


In [162]:
# Dh6p3ID20 -> 5, Dh6p3ID30 -> 1, Dh6p3ID40 -> 0
# voltage
list=[5,0]
fig = go.Figure()
fig.layout.height=500
fig.layout.width=500
fig.layout.legend.x=0
fig.layout.legend.y=1.0
REFERENCE_CAPACITIES_LIST=[np.linspace(0,1,41), np.linspace(0,1,31), REFERENCE_CAPACITIES,REFERENCE_CAPACITIES,REFERENCE_CAPACITIES,REFERENCE_CAPACITIES]
for cell in list:
    cell=int(cell)
    if(cell == 5):
        name = 'ID20'
    elif(cell == 1):
        name = "ID30"
    else:
        name = "ID40"
    window_length = datasets_list[cell]["X_train"].shape[1]
    n_features = datasets_list[cell]["X_train"].shape[2]
    batch_input_shape = (datasets_list[cell]["batch_size"], window_length, n_features)
    n_outputs = datasets_list[cell]["y_train"].shape[-1]
    nf_steps = datasets_list[cell]["y_train"].shape[1]
    y = datasets_list[cell]["y_train"][:, 0, 0]
    idx = y < 2.9
    weights = np.ones_like(y) * 1
    weights[idx] = 2
    batch_size = datasets_list[cell]["batch_size"]
    print(window_length, n_features, batch_input_shape, n_outputs, nf_steps)


    skip = 1000
    test_cell_id_idx = 0
    last_cycle = 1
    current_cycle = 1
    opacity_list = [1, 0.6, 0.3]
    
    for i in range(0, len(datasets_list[cell]["X_test"]), batch_size * skip):
        df_true = pd.DataFrame(datasets_list[cell]["y_test"][i : i + batch_size][:, 0, 1])
    
        last_cycle = current_cycle
        current_cycle = int(datasets_list[cell]["X_test"][i : i + batch_size][0][0][-1]*MAX_CYCLE)

        if last_cycle> current_cycle:
            test_cell_id_idx += 1
        
        cell_id = test_cell_ids[test_cell_id_idx]
        original_df = datasets_list[cell]["original_test"]

        original_df= original_df[original_df.Cycle==current_cycle]
        original_df= original_df[original_df.Cell==cell_id]
        original_df = original_df[original_df.I < MAX_DISCHARGE_CURRENT]
        original_df = original_df[original_df.I > MIN_DISCHARGE_CURRENT]

        original_df['Qd'] = (original_df['Qd']-original_df['Qd'].min())/(original_df['Qd'].max()-original_df['Qd'].min())

        if len(original_df)< 2:
            continue
        q_new = original_df.Qd.values
        t_new = original_df['t'].values
        time_interpolator = interp1d(x = q_new,  y = t_new, fill_value="extrapolate")
    
    
        for j in range(nf_steps):
            df_pred = pd.DataFrame(models[cell].predict(datasets_list[cell]["X_test"][i : i + batch_size], verbose=0)[:, j, 1])
    
            ref_capacities_wrt_nf = REFERENCE_CAPACITIES_LIST[cell][window_length+j:-nf_steps+j-1]

            error_interpolator = interp1d(x = ref_capacities_wrt_nf, 
                                        y = abs(df_pred[0].values-df_true[0].values)*(TEMPERATURE_MAX - TEMPERATURE_MIN),
                                        fill_value="extrapolate"   
                                          )
        
            q_j = q_new[q_new>= min(REFERENCE_CAPACITIES[window_length+j:-nf_steps+j-1])]
            q_j = q_j[q_j<= max(REFERENCE_CAPACITIES[window_length+j:-nf_steps+j-1])]
            t_interp = time_interpolator(REFERENCE_CAPACITIES[window_length:-nf_steps-1])
            if j==0:
                q_j0 = q_j
            error_intp = error_interpolator(q_j)

            fig.add_trace(
                go.Scatter(
                    x=time_interpolator(q_j),
                    y=error_intp,
                    showlegend=False,
                    mode="lines",
                    line_dash="dash",
                    name = "Interpolated Predictions "+name,
                    marker_color=pallete[cell],
                    opacity=opacity_list[j]
                )
            )
        
            fig.add_trace(
                go.Scatter(
                    x=time_interpolator(ref_capacities_wrt_nf),
                    y=abs(df_pred[0].values-df_true[0].values)*(TEMPERATURE_MAX - TEMPERATURE_MIN),
                    showlegend=True,
                    mode="markers",
                    name = f"Predicted (Forecast Horizon={j+1})"+' '+name,
                    marker_color=pallete[cell],
                    marker_symbol = symbol_list[j],
                    marker_size=8,
#                 marker_opacity=opacity_list[j]
                )
            )


# odf = original_df[original_df.Qd > q_j0.min()]
# odf = odf[odf.Qd < q_j.max()]
# fig.add_trace(
# go.Scatter(
#             x=odf['t'],
#                 y=odf['V'],
#                 showlegend=True,
#                 mode="lines",
#                 name = f"{cell_id} Cycle {current_cycle} ",
#                 line_color=pallete[cell+3]
#             )
#         )


fig.update_yaxes(title="Temperature MAE [K]", showgrid=True)
fig.update_xaxes(title="Times (min)", showgrid=True)
fig.update_layout(template="simple_white")

5 4 (11, 5, 4) 2 3
5 4 (32, 5, 4) 2 3


## Epoch Dh6p3ID20e10, Dh6p3ID20e20, Dh6p3ID20

In [167]:
# Dh6p3ID20 -> 5, Dh6p3ID20e10 -> 7, Dh6p3ID20e20 -> 8
# voltage
list=[7,5]
fig = go.Figure()
fig.layout.height=500
fig.layout.width=500
fig.layout.legend.x=0
fig.layout.legend.y=1.0
# REFERENCE_CAPACITIES_LIST=[np.linspace(0,1,41), np.linspace(0,1,31), REFERENCE_CAPACITIES,REFERENCE_CAPACITIES,REFERENCE_CAPACITIES,REFERENCE_CAPACITIES]
for cell in list:
    cell=int(cell)
    if(cell == 5):
        name = '50 epoch'
    elif(cell == 8):
        name = "20 epoch"
    else:
        name = "10 epoch"
    window_length = datasets_list[cell]["X_train"].shape[1]
    n_features = datasets_list[cell]["X_train"].shape[2]
    batch_input_shape = (datasets_list[cell]["batch_size"], window_length, n_features)
    n_outputs = datasets_list[cell]["y_train"].shape[-1]
    nf_steps = datasets_list[cell]["y_train"].shape[1]
    y = datasets_list[cell]["y_train"][:, 0, 0]
    idx = y < 2.9
    weights = np.ones_like(y) * 1
    weights[idx] = 2
    batch_size = datasets_list[cell]["batch_size"]
    print(window_length, n_features, batch_input_shape, n_outputs, nf_steps)


    skip = 1000
    test_cell_id_idx = 0
    last_cycle = 1
    current_cycle = 1
    opacity_list = [1, 0.6, 0.3]
    
    for i in range(0, len(datasets_list[cell]["X_test"]), batch_size * skip):
        df_true = pd.DataFrame(datasets_list[cell]["y_test"][i : i + batch_size][:, 0, 0])
    
        last_cycle = current_cycle
        current_cycle = int(datasets_list[cell]["X_test"][i : i + batch_size][0][0][-1]*MAX_CYCLE)

        if last_cycle> current_cycle:
            test_cell_id_idx += 1
        
        cell_id = test_cell_ids[test_cell_id_idx]
        original_df = datasets_list[cell]["original_test"]

        original_df= original_df[original_df.Cycle==current_cycle]
        original_df= original_df[original_df.Cell==cell_id]
        original_df = original_df[original_df.I < MAX_DISCHARGE_CURRENT]
        original_df = original_df[original_df.I > MIN_DISCHARGE_CURRENT]

        original_df['Qd'] = (original_df['Qd']-original_df['Qd'].min())/(original_df['Qd'].max()-original_df['Qd'].min())

        if len(original_df)< 2:
            continue
        q_new = original_df.Qd.values
        t_new = original_df['t'].values
        time_interpolator = interp1d(x = q_new,  y = t_new, fill_value="extrapolate")
    
    
        for j in range(nf_steps):
            df_pred = pd.DataFrame(models[cell].predict(datasets_list[cell]["X_test"][i : i + batch_size], verbose=0)[:, j, 0])
    
            ref_capacities_wrt_nf = REFERENCE_CAPACITIES[window_length+j:-nf_steps+j-1]

            error_interpolator = interp1d(x = ref_capacities_wrt_nf, 
                                        y = abs(df_pred[0].values-df_true[0].values)*(VOLTAGE_MAX - VOLTAGE_MIN),
                                        fill_value="extrapolate"   
                                          )
        
            q_j = q_new[q_new>= min(REFERENCE_CAPACITIES[window_length+j:-nf_steps+j-1])]
            q_j = q_j[q_j<= max(REFERENCE_CAPACITIES[window_length+j:-nf_steps+j-1])]
            t_interp = time_interpolator(REFERENCE_CAPACITIES[window_length:-nf_steps-1])
            if j==0:
                q_j0 = q_j
            error_intp = error_interpolator(q_j)

            fig.add_trace(
                go.Scatter(
                    x=time_interpolator(q_j),
                    y=error_intp,
                    showlegend=False,
                    mode="lines",
                    line_dash="dash",
                    name = "Interpolated Predictions "+name,
                    marker_color=pallete[cell],
                    opacity=opacity_list[j]
                )
            )
        
            fig.add_trace(
                go.Scatter(
                    x=time_interpolator(ref_capacities_wrt_nf),
                    y=abs(df_pred[0].values-df_true[0].values)*(VOLTAGE_MAX - VOLTAGE_MIN),
                    showlegend=True,
                    mode="markers",
                    name = f"Predicted (Forecast Horizon={j+1})"+' '+name,
                    marker_color=pallete[cell],
                    marker_symbol = symbol_list[j],
                    marker_size=8,
#                 marker_opacity=opacity_list[j]
                )
            )


# odf = original_df[original_df.Qd > q_j0.min()]
# odf = odf[odf.Qd < q_j.max()]
# fig.add_trace(
# go.Scatter(
#             x=odf['t'],
#                 y=odf['V'],
#                 showlegend=True,
#                 mode="lines",
#                 name = f"{cell_id} Cycle {current_cycle} ",
#                 line_color=pallete[cell+3]
#             )
#         )


fig.update_yaxes(title="Voltage MAE [V]", showgrid=True)
fig.update_xaxes(title="Times (min)", showgrid=True)
fig.update_layout(template="simple_white")

5 4 (11, 5, 4) 2 3


ValueError: in user code:

    File "/Users/mac/opt/anaconda3/envs/10701/lib/python3.10/site-packages/keras/engine/training.py", line 1845, in predict_function  *
        return step_function(self, iterator)
    File "/Users/mac/opt/anaconda3/envs/10701/lib/python3.10/site-packages/keras/engine/training.py", line 1834, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "/Users/mac/opt/anaconda3/envs/10701/lib/python3.10/site-packages/keras/engine/training.py", line 1823, in run_step  **
        outputs = model.predict_step(data)
    File "/Users/mac/opt/anaconda3/envs/10701/lib/python3.10/site-packages/keras/engine/training.py", line 1791, in predict_step
        return self(x, training=False)
    File "/Users/mac/opt/anaconda3/envs/10701/lib/python3.10/site-packages/keras/utils/traceback_utils.py", line 67, in error_handler
        raise e.with_traceback(filtered_tb) from None

    ValueError: Exception encountered when calling layer 

In [168]:
print(datasets_list[7]["X_test"].shape)

(9548, 5, 4)
